<a href="https://colab.research.google.com/github/spalacioc05/Proyecto-Modelos1/blob/main/02_preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**02 - Preprocesado**
####***Santiago Palacio Cárdenas***
####***Sarai Restrepo Rodríguez***
####***Natalia Bernal Gutiérrez***

En este notebook se realiza la carga, exploración y preprocesamiento del dataset `train.csv` de la competencia.
Se incluyen imputación de valores faltantes, codificación categórica, transformación de rangos, asignación ordinal y creación de una función general de limpieza para garantizar reproducibilidad.

##**1. Importamos el dataset desde Kaggle**

Se configura el entorno de trabajo para descargar los datos de la competencia directamente desde Kaggle utilizando el archivo `kaggle.json`, que contiene las credenciales de acceso.
Posteriormente, se descargan y descomprimen los archivos del reto `udea-ai-4-eng-20252-pruebas-saber-pro-colombia`, y finalmente se carga el archivo `train.csv` en un DataFrame de Pandas para verificar su tamaño y realizar una inspección inicial de las primeras filas del conjunto de datos.

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.34GB/s]


In [ ]:
!unzip udea*.zip > /dev/null

In [ ]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

z = pd.read_csv("train.csv")
print ("shape of loaded dataframe", z.shape)

shape of loaded dataframe (692500, 21)


In [ ]:
z.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


##**2. Inspección de los datos**


En esta parte se realiza una revisión inicial del conjunto de datos con el objetivo de identificar valores faltantes en cada columna, ya que esto permite conocer qué variables presentan registros vacíos y será la base para definir las estrategias de limpieza e imputación en las siguientes etapas del preprocesamiento.

In [ ]:
data_fixed = z.copy()

In [ ]:
k = data_fixed.isna().sum()
k[k!=0]

,0
E_VALORMATRICULAUNIVERSIDAD,6287
E_HORASSEMANATRABAJA,30857
F_ESTRATOVIVIENDA,32137
F_TIENEINTERNET,26629
F_EDUCACIONPADRE,23178
...,...
F_TIENEAUTOMOVIL,43623
E_PAGOMATRICULAPROPIO,6498
F_TIENECOMPUTADOR,38103
F_TIENEINTERNET.1,26629


In [ ]:
k[k==0]

,0
ID,0
PERIODO_ACADEMICO,0
E_PRGM_ACADEMICO,0
E_PRGM_DEPARTAMENTO,0
E_PRIVADO_LIBERTAD,0
RENDIMIENTO_GLOBAL,0
INDICADOR_1,0
INDICADOR_2,0
INDICADOR_3,0
INDICADOR_4,0


La revisión de valores faltantes muestra que varias columnas presentan registros vacíos, principalmente en variables como:

`E_HORASSEMANATRABAJA`, `F_ESTRATOVIVIENDA`, `F_TIENEAUTOMOVIL`, `F_TIENECOMPUTADOR`, entre otras.

Estas ausencias pueden deberse a información no reportada por los estudiantes o a errores durante la recolección de los datos.
Identificar estas columnas es fundamental, ya que en los pasos posteriores se definirán estrategias de imputación o tratamiento de valores faltantes para garantizar la calidad del conjunto de datos antes del modelado.

##**3. Codificación de la columna *E_PRGM_ACADEMICO***

En esta sección se aplica la técnica de **codificación de etiquetas** *Label Encoding* sobre la columna `E_PRGM_ACADEMICO` que contiene los nombres de los programas académicos cursados por los estudiantes.
El objetivo es transformar estas categorías de texto en valores numéricos enteros, facilitando así su procesamiento por los modelos de aprendizaje automático.

Mediante la función `LabelEncoder` de `scikit-learn`, cada programa académico recibe una **etiqueta numérica única** reemplazando su valor textual por un identificador entero.


In [ ]:
def cleaning_programa_academico(data_fixed):
  label_encoder = LabelEncoder()
  data_fixed['E_PRGM_ACADEMICO'] = label_encoder.fit_transform(data_fixed['E_PRGM_ACADEMICO'])

In [ ]:
cleaning_programa_academico(data_fixed)
data_fixed.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,300,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,249,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,819,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,14,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,907,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


##**4. Codificación de la columna *E_PRGM_DEPARTAMENTO***

De manera similar al paso anterior en esta sección se aplica la técnica de **Label Encoding** sobre la columna `E_PRGM_DEPARTAMENTO`.
Esta variable representa el **departamento** asociado al programa académico del estudiante y este proceso asigna un **número entero único a cada departamento** sustituyendo las etiquetas de texto por valores numéricos para facilitar su uso en el modelado.

In [ ]:
def cleaning_departamento(data_fixed):
  label_encoder = LabelEncoder()
  data_fixed['E_PRGM_DEPARTAMENTO'] = label_encoder.fit_transform(data_fixed['E_PRGM_DEPARTAMENTO'])

In [ ]:
cleaning_departamento(data_fixed)
data_fixed.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,300,4,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,249,3,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,819,4,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,14,26,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,907,1,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


##**5. Limpieza de la columna *E_VALORMATRICULAUNIVERSIDAD***

Se transforma la columna `E_VALORMATRICULAUNIVERSIDAD` que originalmente contenía **rangos de texto**, en valores numéricos representativos de cada rango y se asigna a cada intervalo de matrícula un valor equivalente a su **punto medio** expresado en millones de pesos.

Por ejemplo:

* Entre 2.5 millones y menos de 4 millones → **3.25**,
* Más de 7 millones → **7.0**,
* No pagó matrícula → **0**.

Además, para los registros faltantes `NaN` se genera una muestra aleatoria de una **distribución normal** basada en la media y desviación estándar de los valores existentes, esto permite completar los datos sin distorsionar la distribución original del conjunto.

In [ ]:
def cleaning_matricula(data_fixed):
    valores_matricula = {
        'Más de 7 millones': 7.0,
        'Entre 5.5 millones y menos de 7 millones': 6.25,
        'Entre 4 millones y menos de 5.5 millones': 4.75,
        'Entre 2.5 millones y menos de 4 millones': 3.25,
        'Entre 1 millón y menos de 2.5 millones': 1.75,
        'Entre 500 mil y menos de 1 millón': 0.75,
        'Menos de 500 mil': 0.25,
        'No pagó matrícula': 0
    }

    data_fixed['E_VALORMATRICULAUNIVERSIDAD'] = (data_fixed['E_VALORMATRICULAUNIVERSIDAD'].map(valores_matricula))

    mean = data_fixed['E_VALORMATRICULAUNIVERSIDAD'].mean()
    std = data_fixed['E_VALORMATRICULAUNIVERSIDAD'].std()

    sample = np.random.normal(mean,std,data_fixed['E_VALORMATRICULAUNIVERSIDAD'].isna().sum())

    data_fixed.loc[data_fixed['E_VALORMATRICULAUNIVERSIDAD'].isna(),'E_VALORMATRICULAUNIVERSIDAD'] = sample

In [ ]:
cleaning_matricula(data_fixed)
data_fixed.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,300,4,6.25,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,249,3,3.25,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,819,4,3.25,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,14,26,4.75,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,907,1,3.25,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


##**6. Limpieza de la columna *E_HORASSEMANATRABAJA***

En esta etapa se convierte la columna `E_HORASSEMANATRABAJA`, que originalmente contenía **rangos de texto**, en valores numéricos equivalentes.

Cada categoría se reemplaza por un valor representativo, por ejemplo:

* Menos de 10 horas → **5**,
* Entre 11 y 20 horas → **15.5**,
* Entre 21 y 30 horas → **25.5**,
* Más de 30 horas → **35**,
* 0 → **0**.

Para los valores faltantes `NaN` se generaron muestras aleatorias siguiendo una **distribución normal** basada en la media y desviación estándar de los datos existentes y de esta manera se conservan las características estadísticas del conjunto y se evita introducir sesgos.

In [ ]:
def cleaning_horas_trabajo(data_fixed):

    horas_promedio = {
        '0': 0,
        'Menos de 10 horas': 5,
        'Entre 11 y 20 horas': (11 + 20) / 2,
        'Entre 21 y 30 horas': (21 + 30) / 2,
        'Más de 30 horas': 35
    }

    data_fixed['E_HORASSEMANATRABAJA'] = data_fixed['E_HORASSEMANATRABAJA'].map(horas_promedio)

    mean = data_fixed['E_HORASSEMANATRABAJA'].mean()
    std = data_fixed['E_HORASSEMANATRABAJA'].std()

    sample = np.random.normal(mean, std, data_fixed['E_HORASSEMANATRABAJA'].isna().sum())

    data_fixed.loc[data_fixed['E_HORASSEMANATRABAJA'].isna(), 'E_HORASSEMANATRABAJA'] = sample

In [ ]:
cleaning_horas_trabajo(data_fixed)
data_fixed.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,300,4,6.25,5.0,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,249,3,3.25,0.0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,819,4,3.25,35.0,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,14,26,4.75,0.0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,907,1,3.25,25.5,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


##**7. Limpieza de la columna *F_ESTRATOVIVIENDA***

En esta etapa se procesa la columna `F_ESTRATOVIVIENDA`, que representa el **estrato socioeconómico** de la vivienda del estudiante.
Primero, los valores faltantes `NaN` se reemplazan por la **moda** de la columna, es decir el estrato más frecuente dentro del conjunto de datos y luego se aplica la técnica de **Label Encoding** para transformar las categorías de texto (“Estrato 1”, “Estrato 2”, etc.) en valores numéricos enteros, lo que permite que la variable sea utilizada por los algoritmos de aprendizaje automático.

In [ ]:
def cleaning_estrato(data_fixed):

  moda = data_fixed['F_ESTRATOVIVIENDA'].mode()[0]

  data_fixed['F_ESTRATOVIVIENDA'] = data_fixed['F_ESTRATOVIVIENDA'].fillna(moda)

  label_encoder = LabelEncoder()
  data_fixed['F_ESTRATOVIVIENDA'] = label_encoder.fit_transform(data_fixed['F_ESTRATOVIVIENDA'])

In [ ]:
cleaning_estrato(data_fixed)
data_fixed.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,300,4,6.25,5.0,2,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,249,3,3.25,0.0,2,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,819,4,3.25,35.0,2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,14,26,4.75,0.0,3,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,907,1,3.25,25.5,2,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


##**8. Limpieza de la columna *F_TIENEINTERNET***

En esta sección se trabaja la columna `F_TIENEINTERNET` que es la que indica si el hogar del estudiante cuenta con acceso a internet.
Primero, los valores faltantes `NaN` se reemplazan por la categoría **“Sin información”** para evitar la pérdida de registros y luego se asignan valores numéricos a cada categoría mediante un mapeo lógico:

* Sí → **1**
* No → **0**
* Sin información → **-1**

Esta conversión permite que la variable sea interpretada correctamente por los modelos de aprendizaje, manteniendo la distinción entre respuesta afirmativa, negativa o desconocida.

In [ ]:
def cleaning_internet(data_fixed):
    data_fixed['F_TIENEINTERNET'] = data_fixed['F_TIENEINTERNET'].fillna('Sin Información')
    data_fixed['F_TIENEINTERNET'] = data_fixed['F_TIENEINTERNET'].map({
        'Si': 1,
        'No': 0,
        'Sin Información': -1
    })

In [ ]:
cleaning_internet(data_fixed)
data_fixed.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,300,4,6.25,5.0,2,1,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,249,3,3.25,0.0,2,0,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,819,4,3.25,35.0,2,1,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,14,26,4.75,0.0,3,1,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,907,1,3.25,25.5,2,1,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


##**9. Limpieza de la columna *F_EDUCACIONPADRE***

En esta etapa se normaliza la columna `F_EDUCACIONPADRE`, que indica el **nivel educativo del padre del estudiante**.
Primero, los valores faltantes `NaN` se reemplazan por la categoría **“Sin información”**, y las respuestas como “No sabe” o “No aplica” se sustituyen por **-1**, indicando ausencia de información válida.

Luego, se utiliza la técnica de **codificación ordinal** (*Ordinal Encoding*) para asignar un valor numérico a cada nivel educativo según su jerarquía comoPor:

Primaria incompleta < Primaria completa < Secundaria completa < Técnica < Profesional < Postgrado.

De esta forma, la variable mantiene el **orden natural de los niveles educativos** representando de manera más precisa el grado académico del padre.

In [ ]:
def cleaning_padre(data_fixed):
    data_fixed['F_EDUCACIONPADRE'] = data_fixed['F_EDUCACIONPADRE'].fillna('Sin Información')
    data_fixed['F_EDUCACIONPADRE'] = data_fixed['F_EDUCACIONPADRE'].replace({
        'No sabe': -1,
        'No Aplica': -1,
        'Sin Información': -1
    })
    levels = ['Ninguno',
              'Primaria incompleta',
              'Primaria completa',
              'Secundaria (Bachillerato) incompleta',
              'Secundaria (Bachillerato) completa',
              'Técnica o tecnológica incompleta',
              'Técnica o tecnológica completa',
              'Educación profesional incompleta',
              'Educación profesional completa',
              'Postgrado'
              ]

    mask = data_fixed['F_EDUCACIONPADRE'] != -1

    encoder = OrdinalEncoder(categories=[levels])
    data_fixed.loc[mask, 'F_EDUCACIONPADRE'] = encoder.fit_transform(data_fixed.loc[mask, ['F_EDUCACIONPADRE']])

    data_fixed['F_EDUCACIONPADRE'] = data_fixed['F_EDUCACIONPADRE'].astype(int)

In [ ]:
cleaning_padre(data_fixed)
data_fixed.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,300,4,6.25,5.0,2,1,5,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,249,3,3.25,0.0,2,0,6,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,819,4,3.25,35.0,2,1,4,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,14,26,4.75,0.0,3,1,-1,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,907,1,3.25,25.5,2,1,2,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


##**10. Limpieza de la columna *F_EDUCACIONMADRE***

En esta sección se aplica el mismo procedimiento utilizado en la columna `F_EDUCACIONPADRE` pero ahora sobre la variable `F_EDUCACIONMADRE` que representa el **nivel educativo de la madre del estudiante**.
Primero, los valores faltantes `NaN` se reemplazan por la categoría **“Sin información”**, y las respuestas como “No sabe” o “No aplica” se codifican con el valor **-1**, indicando ausencia de datos válidos.

Posteriormente se emplea un **codificador ordinal** (*OrdinalEncoder*) para transformar los niveles educativos en una escala numérica ordenada, siguiendo la progresión natural desde “Ninguno” hasta “Postgrado”.
De este modo se conserva la relación jerárquica entre los distintos niveles de educación.

In [ ]:
def cleaning_madre(data_fixed):
    data_fixed['F_EDUCACIONMADRE'] = data_fixed['F_EDUCACIONMADRE'].fillna('Sin Información')
    data_fixed['F_EDUCACIONMADRE'] = data_fixed['F_EDUCACIONMADRE'].replace({
        'No sabe': -1,
        'No Aplica': -1,
        'Sin Información': -1
    })
    levels = ['Ninguno',
              'Primaria incompleta',
              'Primaria completa',
              'Secundaria (Bachillerato) incompleta',
              'Secundaria (Bachillerato) completa',
              'Técnica o tecnológica incompleta',
              'Técnica o tecnológica completa',
              'Educación profesional incompleta',
              'Educación profesional completa',
              'Postgrado'
              ]

    mask = data_fixed['F_EDUCACIONMADRE'] != -1

    encoder = OrdinalEncoder(categories=[levels])
    data_fixed.loc[mask, 'F_EDUCACIONMADRE'] = encoder.fit_transform(data_fixed.loc[mask, ['F_EDUCACIONMADRE']])

    data_fixed['F_EDUCACIONMADRE'] = data_fixed['F_EDUCACIONMADRE'].astype(int)



In [ ]:
cleaning_madre(data_fixed)
data_fixed.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,300,4,6.25,5.0,2,1,5,Si,Si,N,No,Si,Si,9,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,249,3,3.25,0.0,2,0,6,Si,No,N,No,Si,No,5,bajo,0.311,0.215,0.292,0.264
2,308367,20203,819,4,3.25,35.0,2,1,4,Si,No,N,No,No,Si,4,bajo,0.297,0.214,0.305,0.264
3,470353,20195,14,26,4.75,0.0,3,1,-1,Si,No,N,No,Si,Si,4,alto,0.485,0.172,0.252,0.190
4,989032,20212,907,1,3.25,25.5,2,1,2,Si,Si,N,No,Si,Si,2,medio-bajo,0.316,0.232,0.285,0.294


##**11. Limpieza de la columna *E_PAGOMATRICULAPROPIO***

En esta sección se normaliza la columna `E_PAGOMATRICULAPROPIO`, que indica si el estudiante pagó su matrícula con recursos propios.
El procedimiento es similar al utilizado en la variable `F_TIENEINTERNET`. Primero se reemplazan los valores faltantes (`NaN`) por la categoría **“Sin información”** y luego se asignan valores numéricos a cada categoría mediante un mapeo:

   * Sí → **1**
   * No → **0**
   * Sin información → **-1**

Esta codificación permite que la variable se interprete de forma cuantitativa y sea utilizada directamente en los modelos de aprendizaje.


In [ ]:
def cleaning_pago(data_fixed):
    data_fixed['E_PAGOMATRICULAPROPIO'] = data_fixed['E_PAGOMATRICULAPROPIO'].fillna('Sin Información')
    data_fixed['E_PAGOMATRICULAPROPIO'] = data_fixed['E_PAGOMATRICULAPROPIO'].map({
        'Si': 1,
        'No': 0,
        'Sin Información': -1
    })

In [ ]:
cleaning_pago(data_fixed)
data_fixed.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,300,4,6.25,5.0,2,1,5,Si,Si,N,0,Si,Si,9,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,249,3,3.25,0.0,2,0,6,Si,No,N,0,Si,No,5,bajo,0.311,0.215,0.292,0.264
2,308367,20203,819,4,3.25,35.0,2,1,4,Si,No,N,0,No,Si,4,bajo,0.297,0.214,0.305,0.264
3,470353,20195,14,26,4.75,0.0,3,1,-1,Si,No,N,0,Si,Si,4,alto,0.485,0.172,0.252,0.190
4,989032,20212,907,1,3.25,25.5,2,1,2,Si,Si,N,0,Si,Si,2,medio-bajo,0.316,0.232,0.285,0.294


##**12. Conversión de la columna objetivo *RENDIMIENTO_GLOBAL***

En esta etapa se transforma la variable `RENDIMIENTO_GLOBAL` que representa el **nivel de desempeño académico del estudiante** en una variable numérica que puede ser procesada por los modelos de aprendizaje automático.

Se asignan valores enteros a cada categoría de rendimiento de acuerdo con su jerarquía:

* bajo → **0**
* medio-bajo → **1**
* medio-alto → **2**
* alto → **3**

Este mapeo convierte la variable de texto en una escala ordinal coherente, manteniendo el orden lógico de los niveles de desempeño.

In [ ]:
def cleaning_rendimiento(data_fixed):
  rmap = {'alto': 3, 'bajo':0, 'medio-bajo':1, 'medio-alto':2}
  data_fixed['RENDIMIENTO_GLOBAL'] = data_fixed['RENDIMIENTO_GLOBAL'].map(rmap)

In [ ]:
cleaning_rendimiento(data_fixed)
data_fixed.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,300,4,6.25,5.0,2,1,5,Si,Si,N,0,Si,Si,9,2,0.322,0.208,0.310,0.267
1,645256,20212,249,3,3.25,0.0,2,0,6,Si,No,N,0,Si,No,5,0,0.311,0.215,0.292,0.264
2,308367,20203,819,4,3.25,35.0,2,1,4,Si,No,N,0,No,Si,4,0,0.297,0.214,0.305,0.264
3,470353,20195,14,26,4.75,0.0,3,1,-1,Si,No,N,0,Si,Si,4,3,0.485,0.172,0.252,0.190
4,989032,20212,907,1,3.25,25.5,2,1,2,Si,Si,N,0,Si,Si,2,1,0.316,0.232,0.285,0.294


##**13. Función de limpieza completa del dataset**

En esta sección se define la función principal `data_cleaning()` encargada de centralizar todo el proceso de limpieza y transformación del dataset, esta función importa las librerías necesarias y llama secuencialmente a cada una de las funciones de limpieza definidas anteriormente, las cuales realizan las transformaciones específicas en las columnas correspondientes.

Cada función individual trata una variable del conjunto de datos aplicando técnicas como **codificación de etiquetas**, **imputación de valores faltantes**, **normalización de rangos** y **conversión de categorías textuales en valores numéricos**.
De esta manera, se obtiene un nuevo dataset compuesto únicamente por valores numéricos, completamente preparado para la fase de modelado.

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

def data_cleaning(data):

    data_fixed = data.copy()
    cleaning_programa_academico(data_fixed)
    cleaning_departamento(data_fixed)
    cleaning_matricula(data_fixed)
    cleaning_horas_trabajo(data_fixed)
    cleaning_estrato(data_fixed)
    cleaning_internet(data_fixed)
    cleaning_padre(data_fixed)
    cleaning_madre(data_fixed)
    cleaning_pago(data_fixed)

    if 'RENDIMIENTO_GLOBAL' in data_fixed.columns:
      cleaning_rendimiento(data_fixed)

    return data_fixed

In [ ]:
data_fixed = data_cleaning(z)
data_fixed.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,300,4,6.25,5.0,2,1,5,Si,Si,N,0,Si,Si,9,2,0.322,0.208,0.310,0.267
1,645256,20212,249,3,3.25,0.0,2,0,6,Si,No,N,0,Si,No,5,0,0.311,0.215,0.292,0.264
2,308367,20203,819,4,3.25,35.0,2,1,4,Si,No,N,0,No,Si,4,0,0.297,0.214,0.305,0.264
3,470353,20195,14,26,4.75,0.0,3,1,-1,Si,No,N,0,Si,Si,4,3,0.485,0.172,0.252,0.190
4,989032,20212,907,1,3.25,25.5,2,1,2,Si,Si,N,0,Si,Si,2,1,0.316,0.232,0.285,0.294


La función `data_cleaning()` consolida todas las etapas del preprocesamiento en una sola rutina, permitiendo reproducir el proceso de limpieza de manera eficiente y consistente sobre cualquier conjunto de datos similar.
El resultado final es un **dataset depurado y homogéneo** donde cada variable está representada en formato numérico, listo para ser utilizado en el entrenamiento de modelos predictivos.

## **14. Dificultades y posibles alteraciones en los datos**

Durante el proceso de limpieza y transformación, se detectaron ligeras **variaciones en la distribución original de algunas variables** particularmente en aquellas que fueron codificadas o imputadas con valores estimados.

In [ ]:
data_fixed.shape

(692500, 21)

### **14.1. Incremento de registros en “Estrato 2”**

In [ ]:
estrato2_procesado = (data_fixed['F_ESTRATOVIVIENDA'] == 1).sum()
print(f"Total de registros con 'Estrato 2' (procesado): {estrato2_procesado}")
estrato2_original = (z['F_ESTRATOVIVIENDA'] == 'Estrato 2').sum()
print(f"Total de registros con 'Estrato 2' (original): {estrato2_original}")

Total de registros con 'Estrato 2' (procesado): 264808
Total de registros con 'Estrato 2' (original): 232671


El **incremento de registros en “Estrato 2”** de 232,671 a 264,808 evidencia una leve sobrerrepresentación generada por la imputación mediante la moda en la columna `F_ESTRATOVIVIENDA`. Este procedimiento, aunque útil para eliminar valores faltantes introduce un posible **sesgo hacia la categoría dominante**, alterando ligeramente la proporción real de los estratos socioeconómicos.

### **14.2. Aumento de valores únicos en variables numéricas imputadas**

In [ ]:
numeric_cols = ['E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA']
for col in numeric_cols:
    var_original = z[col].dropna().astype(str).nunique()
    var_processed = data_fixed[col].nunique()
    print(f"{col}: valores únicos originales = {var_original}, procesados = {var_processed}")

E_VALORMATRICULAUNIVERSIDAD: valores únicos originales = 8, procesados = 6295
E_HORASSEMANATRABAJA: valores únicos originales = 5, procesados = 30862


El **aumento de valores únicos en las variables numéricas imputadas** `E_VALORMATRICULAUNIVERSIDAD` pasó de 8 a 6,295 y `E_HORASSEMANATRABAJA` de 5 a 30,862. Muestra que el uso de una **distribución normal aleatoria** para reemplazar los valores nulos incrementó la dispersión de los datos. Este efecto puede añadir **ruido estadístico** afectando el comportamiento de los modelos que dependen de la varianza de estas variables.

### **14.3. Registros sin información en F_EDUCACIONPADRE**

In [ ]:
info_faltante_original = []

no_aplica = (z['F_EDUCACIONPADRE'] == 'No Aplica').sum()
sin_respuesta = z['F_EDUCACIONPADRE'].isna().sum()
no_sabe = (z['F_EDUCACIONPADRE'] == 'No sabe').sum()

info_faltante_original.extend([no_aplica, sin_respuesta, no_sabe])

info_faltante_total = (data_fixed['F_EDUCACIONPADRE'] == -1).sum()

print(f"Registros sin información en el dataset original: {info_faltante_original}")
print(f"Total de registros sin información (procesado): {info_faltante_total}")

Registros sin información en el dataset original: [np.int64(9229), np.int64(23178), np.int64(16592)]
Total de registros sin información (procesado): 48999


Los **48,999 registros sin información en `F_EDUCACIONPADRE`** agrupan los casos en los que los estudiantes respondieron “No sabe”, “No aplica” o dejaron el campo vacío. Aunque se codificaron con el valor -1 para mantener la completitud del dataset esta agrupación puede **afectar la distribución y las correlaciones** internas, especialmente en análisis que relacionen la educación de los padres con el rendimiento académico.

### **14.4. Ligero incremento de la moda en variables categóricas imputadas**

In [ ]:
categorical_cols = ['F_ESTRATOVIVIENDA', 'F_TIENEINTERNET']
for col in categorical_cols:
    top_original = z[col].value_counts().idxmax()
    top_processed = data_fixed[col].value_counts().idxmax()
    print(f"Columna: {col}")
    print(f"Categoría más frecuente (original): {top_original}")
    print(f"Categoría más frecuente (procesada): {top_processed}\n")

Columna: F_ESTRATOVIVIENDA
Categoría más frecuente (original): Estrato 2
Categoría más frecuente (procesada): 1

Columna: F_TIENEINTERNET
Categoría más frecuente (original): Si
Categoría más frecuente (procesada): 1



Por último, se detectó un **ligero incremento en la moda de algunas variables categóricas imputadas**, como `F_TIENEINTERNET` y `F_ESTRATOVIVIENDA`. Este efecto se debe a la sustitución de valores faltantes por la categoría más frecuente, lo que genera un **pequeño sesgo en las proporciones** y reduce marginalmente la diversidad de las respuestas originales.
